# 1. is vs ==

In [ ]:
a = [1, 2, 3]
b = [1, 2, 3]
print id(a)
print id(b)

In [ ]:
print a == b

In [ ]:
print a is b

In [ ]:
c = a
print id(c) == id(a)
print c is a

## 1.1. None, True, False

In [ ]:
a = None
b = None
print a is b

In [ ]:
print id(None)
print id(a)

In [ ]:
a = True
print id(a), id(True)


In [ ]:
print dir(None)
print type(None)
print type(True)
print type(False)

## 1.2. Sprawdzanie wartości negatywnych

Takie sprawdzenie nie będzie działać:

In [ ]:
a = {}
if a is {}:
    print "is empty"

A takie będzie:

In [ ]:
if a == {}:
    print "is equal to empty"

Najlepiej jednak używać skróconej formy korzystającej z domyślnej konwersji do bool'a:

In [ ]:
def is_empty(item):
    if not item:
        print "is empty"
        return True
    return False

a = {}
b = []
print is_empty(a)
print is_empty(b)

To nie ma zastosowania przy argumentach domyślnych:

In [ ]:
def calc_count():
    print "Calculated count to 10"
    return 10


def check_sth_important(count=None):
    if count is None:
        count = calc_count()
    return count == 100


def check_sth_important_sec(count=None):
    if not count:
        count = calc_count()
    return count == 100


print check_sth_important(0)
print check_sth_important_sec(0)
        
    

## 1.3. Porównywanie obiektów

Sprawdźmy mechanizm porównywania dla słownika:

In [ ]:
a = {"burak": "cukrowy", u"samochód": "terenowy"}
b = {u"samochód": "terenowy", "burak": "cukrowy"}
c = {u"samochód": "terenowy", "burak": "niecukrowy"}
d = {u"samochód": "terenowy", "burak": "cukrowy", "spinacz": "biurowy"}
print a == b
print a == c
print a == d

In [ ]:
print dir(a)

In [ ]:
print a.__eq__
print a.__eq__(b)

https://github.com/python/cpython/blob/2.7/Objects/dictobject.c#L1954

Przykład z życia wzięty (czy musimy nadpisywać?):

In [ ]:
def update_resource(resource, params):
    new = resource.copy()
    new.update(params)
    if resource == new:
        return
    print "Updating... {}".format(params)


resource = {"id": 1, "name": "sth", "desc": "Fst res"}
update_resource(resource, {"name": "sth2"})
update_resource(resource, {"name": "sth"})


Inne wbudowane typy także mają zaimplementowane intuicyjne mechanizmy porównywania.

## 1.4. Wartości boolowskie obiektów

Każdy typ można zrzutować do bool'a:

In [ ]:
print bool(0)
print bool(1)

print bool({})
print bool({"a": 1})

print bool([])
print bool([1])

print bool(set([]))
print bool(set([1]))


In [ ]:
class ParityValue(object):
    def __init__(self, v):
        self.value = v
        
print bool(ParityValue(3))

In [ ]:
def pv_nonzero(self):
    if self.value % 2 == 0:
        return True
    return False

ParityValue.__nonzero__ = pv_nonzero

print bool(ParityValue(3))

## 1.5. Podsumowanie

* None, True, False to singletony
* Zalecana jest zwięzła konstrukcja "if sth:"
* PEP8 wymaga żeby z None zawsze używać operatora "is"
* Każdy obiekt w pythonie mapuje się na jakąś wartość boolowską
* To, jak dany obiekt mapuje się na wartość boolowską można zmienić
* Porównania wbudowanych typów są intuicyjne, resztę można zmienić metodą __eq__


# 2. Więcej o For

## 2.1. enumerate

Funkcja 'enumerate' przydaje się przy iteracji:

In [ ]:
from collections import namedtuple


Book = namedtuple('Book', ['author', 'name'])

confiscated_books = [
    Book(u'Cat-Mackiewicz', u'Europa in flagranti'),
    Book(u'Pudzian', u'Jestem silny'),
    Book(u'Kopernik', u'O obrotach ciał niebieskich')
]

forbidden_authors = set(['Pudzian'])


def find_forbidden(books, forbidden_authors):
    indexes = []
    for i, book in enumerate(books):
        if book.author in forbidden_authors:
            indexes.append(i)
    return indexes

find_forbidden(confiscated_books, forbidden_authors)

In [ ]:
[i for i, b in enumerate(confiscated_books) if b.author in forbidden_authors]

## 2.2. for...else

In [ ]:
for i in [2, 4, 6]:
    if i % 2 != 0:
        break
else:
    print "All are ok!"

Przydatne przy problemach typu 'all or nothing':

In [ ]:
Chunk = namedtuple('Chunk', ['body', 'parity'])

chunks = [
    Chunk('Something is no yes', 1),
    Chunk(' but I will survivex', 1),
    Chunk(' I will do it.', 0)
]


def is_chunk_correct(chunk):
    return (len(chunk.body) + chunk.parity) % 2 == 0


def recreate_messege(chunks):
    return "".join([ch.body for ch in chunks])


def notify_failure(index):
    print("Failure on chunk: {0}".format(index))


def from_chunks(chunks):
    for i, chunk in enumerate(chunks):
        if not is_chunk_correct(chunk):
            break
    else:
        return recreate_messege(chunks)
    notify_failure(i)


from_chunks(chunks)


## 2.3. Wyrażenia listowe vs zwykły for

Przypomnienie:

In [ ]:
nums = [2, 3, 4, 5]

nums_sq = [n**2 for n in nums]
print(nums_sq)

print(id(nums) == id(nums_sq))

Zagnieżdżone wyrażenia listowe:

In [ ]:
deeper = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
[i for inner in deeper for i in inner]

Wyrażenia listowe:
* "bezstanowa" transformacja danych 

In [ ]:
words = ["kajak", "oko", "maroko", "wikary"]
[w for w in words if w == w[::-1]]

Zwykły for:
* kiedy wykonanie następnego obrotu pętli zależy od poprzedniego - "stanowa" pętla
* kiedy operacji w jednym obrocie pętli jest dużo (wyrażenie listowe staje się wtedy nieczytelne)

In [ ]:
a, b = 1, 1
n = 6
for i in xrange(2, n):
    a, b = a + b, a
print(a)

## ZADANIE 1

Przefiltruj poniższą listę używając wyrażeń listowych i znajdując tylko obiekty w stanie 'OK':

In [ ]:
items = [
    [{"name": "chair", "state": "OK"}, {"name": "table", "state": "BAD"}],
    [{"name": "TV", "state": "BAD"}, {"name": "sofa", "state": "BAD"}],
    [{"name": "bed", "state": "OK"}, {"name": "desk", "state": "BAD"}]
]

## ZADANIE 2

Ulepsz poniższy kod, wykorzystując prezentowane w tym rozdziale konstrukcje:

In [ ]:
from PIL import Image

X_POINTS = 1000
Y_POINTS = 1000


def mandelbrot(z, maxiter):
    """
    Calculate single point.
    """
    c = z
    for n in xrange(maxiter):
        if abs(z) > 2:
            return n
        z = z * z + c
    return maxiter


def mandelbrot_set(x_min, x_step, x_cnt, y_min, y_step, y_cnt):
    """
    Calculate X_POINTS*Y_POINTS points on complex plane to see
    which ones belong to Mandelbrot set.
    x_min: starting point on X axis
    x_step: distance between two consecutive poinst on X axis
    x_cnt: number of poinst function calculates on single line
    """
    x_points = []
    for i in xrange(x_cnt):
        n_point = x_min + x_step * i
        x_points.append(n_point)
        
    y_points = []
    for i in xrange(y_cnt):
        n_point = y_min + y_step * i
        y_points.append(n_point)
    
    m_set = []
    for real in x_points:
        for img in y_points:
            point = mandelbrot(complex(real, img), 256)
            m_set.append(point)
    
    return m_set


def to_rgb(m_set):
    rgb = []
    for i in m_set:
        rgb.append((i, 0, 0))
    return rgb


def run():
    outcome = mandelbrot_set(-2.0, 0.005, X_POINTS, -2.0, 0.005, Y_POINTS)
    outcome = to_rgb(outcome)
    im = Image.new('RGB', (X_POINTS, Y_POINTS))
    im.putdata(outcome)
    im.save("mandelbrot.png", "PNG")


run()

In [ ]:
import random
from IPython.display import HTML, display

display(HTML(('<img src="mandelbrot.png?{}" ' +
             'alt="Mandelbrot" ' +
             'height="100">').format(random.randint(0,2e9))))